In [ ]:
pip install ciso8601

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import random
import copy
import tarfile
import pandas as pd
from datetime import datetime, timedelta
import ciso8601
import matplotlib.pyplot as plt
import random

In [2]:
df = pd.read_csv("C:/Users/user/Desktop/AML/thesis/data/CC_LCL-FullData.csv")

In [3]:
# Renaming the power column and convert string to datetime and convert "Null" string to Nan
df.rename(columns={'KWH/hh (per half hour) ': 'load'},inplace=True)
df['DateTime'] = df['DateTime'].apply(lambda x:  ciso8601.parse_datetime(x[0:-8]))
df.load = pd.to_numeric(df.load,errors = 'coerce')

In [7]:
x = df.drop_duplicates(['LCLid','DateTime']).groupby(['LCLid']).mean()

C:\Users\user\AppData\Local\Temp\ipykernel_13756\2371656722.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  x = df.drop_duplicates(['LCLid','DateTime']).groupby(['LCLid']).mean()


In [12]:
y = df.drop_duplicates(['LCLid','DateTime']).groupby(['LCLid']).median()

C:\Users\user\AppData\Local\Temp\ipykernel_13756\1387506559.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  y = df.drop_duplicates(['LCLid','DateTime']).groupby(['LCLid']).median()


In [11]:
x.load.max()

2.111239230527941

In [13]:
y.load.max()

1.925

In [ ]:
df.head()

In [ ]:
len(df['LCLid'].unique())

In [ ]:
# Keeping only standard tariff customers
df_std = df[df.stdorToU=='Std']

In [ ]:
df_std.head()

In [ ]:
# finding which ids have data from the entirety of the 2013
start = datetime(year=2013, month = 1, day = 1, hour = 0, minute= 0, second= 0)
end = datetime(year=2013, month = 12, day = 31, hour = 23, minute= 30, second= 0)
ids = set(df_std[df_std['DateTime']==start].LCLid) & set(df_std[df_std['DateTime']==end].LCLid)

In [ ]:
# Keeping data only for the year of 2013 and for the ids that have data for the entire 2013
df_std_2013 = df_std[(df_std.DateTime>=start)&(df_std.DateTime<=end)&(df_std.LCLid.isin(ids))]

In [ ]:
# Dropping the necesarry duplicates and resetting index
df_std_2013.drop_duplicates(['LCLid','DateTime'],keep = 'first',inplace = True)
df_std_2013.reset_index(inplace=True, drop=True)

In [ ]:
df_std_2013.head()

In [ ]:
len(df_std_2013.LCLid.unique())

In [ ]:
# non zero load 
ids2 = set(df_std_2013.LCLid.unique()) - set(df_std_2013[(df_std_2013.load==0)]['LCLid'].unique())

In [ ]:
len(ids2)

In [ ]:
df_std_2013 = df_std_2013[df_std_2013.LCLid.isin(ids2)]
df_std_2013.head()

In [ ]:
# finding those participants for which the difference between two consectuive rows' datetimes is more than an hour or less than half an hour
names = []
x = df_std_2013.groupby('LCLid')['DateTime'].diff()
y = x[(x > timedelta(hours=1))|(x < timedelta(minutes = 30))]

for idx in y.index:
    names.append(df_std_2013.loc[idx,'LCLid'])

In [ ]:
df_std_2013_cleaned = df_std_2013[~df_std_2013.LCLid.isin(set(names))]
df_std_2013_cleaned.head(2)

In [ ]:
# Inserting rows at appropriate locations
# Calculate time difference between consecutive rows for each ID
df_std_2013_cleaned['time_diff'] = df_std_2013_cleaned.groupby('LCLid')['DateTime'].diff()


# Filter rows where time difference is greater than 30 minutes
rows_to_insert = df_std_2013_cleaned[df_std_2013_cleaned['time_diff'] > timedelta(minutes=30)].copy()

# Create new rows to be inserted
rows_to_insert['DateTime'] = rows_to_insert['DateTime'] - timedelta(minutes=30)

# Calculate the load as the average of previous and following rows
rows_to_insert['load'] = (rows_to_insert['load'].shift() + rows_to_insert['load'].shift(-1)) / 2

# Concatenate original DataFrame and rows to be inserted
df_std_2013_cleaned = pd.concat([df_std_2013_cleaned, rows_to_insert]).sort_values(['LCLid', 'DateTime']).reset_index(drop=True)



In [ ]:
# Fill load Nans with average of the previous and latter. It was checked that for no id, the first or the last load value is Nan. Thus, no mixup between IDs
# Also dropping the time_diff column
df_std_2013_cleaned['load'] = df_std_2013_cleaned['load'].fillna((df_std_2013_cleaned['load'].ffill() + df_std_2013_cleaned['load'].bfill()) / 2)
df_std_2013_cleaned.drop('time_diff', axis=1,inplace = True)

In [ ]:
# This shows that ony time diff is 30 mins
diff = df_std_2013_cleaned.groupby(['LCLid']).DateTime.diff()
set(diff[~(diff==timedelta(minutes = 30))])

In [ ]:
df_acorn = pd.read_csv("C:/Users/user/Desktop/AML/thesis/data/UK power networks data/informations_households.csv", encoding='latin-1')

In [ ]:
# Adding the Acorn column
df_merged = df_std_2013_cleaned.merge(df_acorn[['LCLid', 'Acorn']], on='LCLid', how='left')

In [ ]:
df_merged.groupby(['Acorn','LCLid']).count().groupby(['Acorn']).count()

In [ ]:
# Getting all ids for a given Acorn Group
df_id_acorn = df_merged[['LCLid','Acorn']].drop_duplicates(['LCLid','Acorn'])
acorns = df_id_acorn['Acorn'].unique()
acorns.sort()
dict = {}
for i in acorns:
    dict[i] = df_id_acorn[df_id_acorn.Acorn==i].LCLid.unique()

In [ ]:
# # Sampling 45% per group 
# random.seed(0)
# final_ids = []
# for i in dict:
#     samples = np.random.choice(dict[i], size=int(0.45*len(dict[i])), replace=False)
#     final_ids.extend(samples.tolist())
# final_ids.sort()

In [ ]:
final_ids = df_id_acorn.LCLid.unique()

In [ ]:
min(final_ids)

In [ ]:
df_ids = df_merged[df_merged.LCLid.isin(final_ids)].sort_values(['LCLid', 'DateTime', 'Acorn'], ascending=[True, True, True]).reset_index(drop = True)

In [ ]:
df_ids.head()

In [ ]:
# Create a new dataframe to store the aggregated rows
df_aggregated = pd.DataFrame()

# Assign the LCLid, stdorToU, and DateTime from the first row of every two rows
df_aggregated['LCLid'] = df_ids['LCLid'].iloc[::2].values
df_aggregated['stdorToU'] = df_ids['stdorToU'].iloc[::2].values
df_aggregated['DateTime'] = df_ids['DateTime'].iloc[::2].values

# Combine the Load values of every two consecutive rows
df_aggregated['load'] = df_ids['load'].iloc[::2].values + df_ids['load'].iloc[1::2].values

#Assign the Acorn from the first row of every two rows
df_aggregated['Acorn'] = df_ids['Acorn'].iloc[::2].values

df_aggregated.sort_values(['LCLid', 'DateTime', 'Acorn'], ascending=[True, True, True]).reset_index(drop = True)

In [ ]:
df_aggregated.groupby(['LCLid','Acorn']).count().groupby(['Acorn']).count()

In [ ]:
# This shows that ony time diff is 1 hour
diff = df_aggregated.groupby(['LCLid']).DateTime.diff()
set(diff)

In [ ]:
from datetime import datetime

# Create a datetime object
dt = datetime.now()

# Get the day of the week (Monday is 0 and Sunday is 6)
day_of_week = dt.weekday()

# Get the hour in 24-hour format
hour = dt.strftime("%H")

# Print the day of the week and hour
print("Day of the week:", day_of_week)
print("Hour:", hour)

In [ ]:
# adding the day of the week and hour column
df_aggregated['Weekday'] = df_aggregated.DateTime.apply(lambda x: x.weekday())
df_aggregated['Hour'] = df_aggregated.DateTime.apply(lambda x: int(x.strftime("%H")))

In [ ]:
df_aggregated.head(1)

In [ ]:
df_weather = pd.read_csv('weather_hourly_darksky.csv')
df_weather.rename(columns={'time': 'DateTime'},inplace=True)
df_weather['DateTime'] = df_weather['DateTime'].apply(lambda x:  ciso8601.parse_datetime(x))
df_weather.sort_values(['DateTime'],inplace = True)
df_weather.reset_index(drop = True,inplace = True)
df_weather.head(2)

idx = df_weather[df_weather['DateTime'].diff()==timedelta(hours = 3)].index[0]

In [ ]:
df_weather = pd.read_csv('weather_hourly_darksky.csv')
df_weather.rename(columns={'time': 'DateTime'},inplace=True)
df_weather['DateTime'] = df_weather['DateTime'].apply(lambda x:  ciso8601.parse_datetime(x))
df_weather.sort_values(['DateTime'],inplace = True)
df_weather.reset_index(drop = True,inplace = True)
df_weather.head(2)

idx = df_weather[df_weather['DateTime'].diff()==timedelta(hours = 3)].index[0]
print(idx)


#inserting missing rows. only one pair of consecutive rows for which diff is more than 1 (3hrs). same apparent temp
# Create the new DataFrame with new rows
row1 = copy.deepcopy(df_weather.iloc[idx-1])
row1['DateTime'] = row1['DateTime'] + timedelta(hours = 1)
row2 = copy.deepcopy(df_weather.iloc[idx-1])
row2['DateTime'] = row2['DateTime'] + timedelta(hours = 2)

# Concatenate the new rows with the original DataFrame and sort the index
df_weather = pd.concat([df_weather.iloc[:idx], pd.DataFrame([row1]),pd.DataFrame([row2]), df_weather.iloc[idx:]]).reset_index(drop=True)
df_weather.sort_values(['DateTime'],inplace = True)
df_weather.reset_index(drop = True,inplace = True)


In [ ]:
# Adding apparent temperature
df_aggregated = pd.merge(df_aggregated, df_weather[['DateTime','apparentTemperature']], on='DateTime', how='left')
df_aggregated.head(1)


In [ ]:
# Step 3: Group the dataframe by LCLid and weekday/weekend category
grouped = df_aggregated.groupby(['LCLid', df_aggregated['DateTime'].dt.weekday.isin([5, 6])])

# Step 4: Create a new column with default values for AVG4D
df_aggregated['AVG4D'] = pd.Series(dtype='float64')

# Step 5: Define a function to calculate AVG4D using rolling window
def calculate_avg4d(group):
    # Identify current day's category
    is_weekend = group.name[1]

    # Calculate rolling average load over a window of 4 days
    rolling_avg = group['load'].rolling(window=4, min_periods=1).mean()
    
    # Assign rolling average values to AVG4D column
    group['AVG4D'] = rolling_avg
    
    return group

# Step 6: Apply the function to each group
grouped.apply(calculate_avg4d)

In [ ]:
df_weather.head(1)

In [ ]:
import numpy as np
import pandas as pd

# Sort the dataframe by DateTime in ascending order
df_aggregated = df_aggregated.sort_values('DateTime')

# Create a boolean mask to identify weekdays and weekends
is_weekday = ~(df_aggregated['DateTime'].dt.dayofweek.isin([5,6]))

# Create rolling windows for weekdays and weekends, considering the same time and LCLid
weekday_window = df_aggregated[is_weekday].groupby(['LCLid', df_aggregated['DateTime'].dt.time])['load'].rolling(4,min_periods = 1)
weekend_window = df_aggregated[~is_weekday].groupby(['LCLid', df_aggregated['DateTime'].dt.time])['load'].rolling(4,min_periods = 1)

# Compute the average load for each rolling window
df_aggregated['AVG4D'] = np.nan
df_aggregated.loc[is_weekday, 'AVG4D'] = weekday_window.mean().reset_index(level=[0, 1], drop=True)
df_aggregated.loc[~is_weekday, 'AVG4D'] = weekend_window.mean().reset_index(level=[0, 1], drop=True)

df_aggregated.sort_values(['LCLid', 'DateTime', 'Acorn'], ascending=[True, True, True],inplace = True)
df_aggregated.reset_index(drop = True,inplace = True)


In [ ]:
len(df_aggregated['LCLid'].unique())

In [ ]:
df_aggregated

In [ ]:
# tempcluster
import numpy as np
from sklearn.cluster import KMeans

# Create a sample series
series = df_aggregated['apparentTemperature']

# Reshape the series to a 2D array
X = series.values.reshape(-1, 1)

# Apply K-means clustering with K = 2
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans.fit(X)

# Get the cluster labels for each point
labels = kmeans.labels_

# Print the cluster labels
print(labels)


In [ ]:
df_aggregated['TempCluster'] = labels

In [ ]:
df_final = df_aggregated

In [ ]:
# import pickle
# with open('final_ids.pkl', 'wb') as f:
#     pickle.dump(final_ids, f)  # Save ids to a pickle file

# with open('df_final.pkl', 'wb') as f:
#     pickle.dump(df_final, f)  # Save DataFrame to a pickle file